### Fetch input CSV from Bronze /Files

In [ ]:
df = spark.read.format("csv") \
    .option("header","true") \
    .load("Files/raw_data.csv")

### Transform the Dataframe before output

In [ ]:
from pyspark.sql.functions import current_timestamp

df = df.withColumn("processed_date_time", current_timestamp())
display(df)

### Write CSV to Bronze /Files

In [ ]:
# Write back to Bronze Files as CSV
df.write.mode('overwrite') \
    .csv('Files/output_csv_Bronze.csv')

### Write CSV to Silver /Files

In [ ]:
# Function to get mount path for any LH in the workspace given its name
def get_mount_path(lakehouse_name):
    mnt_point = f'/mnt/mnt_{lakehouse_name}'
    mssparkutils.fs.mount(lakehouse_name, mnt_point)
    return f'file:{mssparkutils.fs.getMountPath(mnt_point)}'

mount_path = get_mount_path('Silver_LH')
df.write.mode('overwrite') \
    .csv(f'{mount_path}/Files/output_csv_Silver.csv')

### Write Delta to Bronze /Tables

In [ ]:
df.write.format("delta").mode("overwrite") \
    .saveAsTable('output_table_Bronze')

### Write Delta to Silver /Tables

In [ ]:
df.write.format("delta").mode("overwrite") \
    .saveAsTable('Silver_LH.output_table_Silver')